In [4]:
import fsspec
import pandas as pd

In [5]:
host='carnation.dbs.umt.edu'
port=2229
user=''
pw=''


fs = fsspec.filesystem('ssh', host=host, port=port, username=user, password=pw)

gaierror: [Errno 8] nodename nor servname provided, or not known

In [3]:
with fs.open('./YNP_RNA/raw_data/fastqs/Mguttatus_256_v2.0.gene_exons.gff3') as f:
    df = pd.read_csv(f, sep='\t', names=['chromosome','source','feature_type','start_pos','end_pos', 'score','strand','frame','attribute'],skiprows=2)

with fs.open('./YNP_RNA/raw_data/fastqs/Mguttatus_256_v2.0.gene_exons.gff3') as f:
    df1 = pd.read_csv(f, sep='\t', names=['chromosome','source','feature_type','start_pos','end_pos', 'score','strand','frame','attribute'],skiprows=2)

# string split the attribute tab 

def append_ah_suffix(df: pd.DataFrame, suffix: str):
    ndf = df[df['feature_type'].isin(['mRNA', 'gene'])].attribute.str.split(r'(?<=Name)(.*?)(?=;)',expand=True, regex=True)
    df['attribute'][df['feature_type'].isin(['mRNA', 'gene'])] =  ndf.iloc[:,0] + ndf.iloc[:,1] + suffix + ndf.iloc[:,2]
    return df 

df['chromosome'] = df['chromosome'] + '_AHQT'
df1['chromosome'] = df1['chromosome'] + '_AHQN'

df = append_ah_suffix(df, suffix='_AHQT')
d1 = append_ah_suffix(df1, suffix = '_AHQN')


frames = [df, df1]
df_merge = pd.concat(frames)

NameError: name 'fs' is not defined

In [92]:
df[df['feature_type'].isin(['mRNA', 'gene'])].attribute.iloc[0]



'ID=Migut.A00001.v2.0;Name=Migut.A00001;ancestorIdentifier=mgv1a001726m.g.IM62.v1.1; Name=AHQT'

In [99]:
ndf

,0,1,2
0,ID=Migut.A00001.v2.0;Name,=Migut.A00001,;ancestorIdentifier=mgv1a001726m.g.IM62.v1.1; ...
1,ID=Migut.A00001.1.v2.0;Name,=Migut.A00001.1,;pacid=28939341;longest=1;Parent=Migut.A00001....
23,ID=Migut.A00002.v2.0;Name,=Migut.A00002,;ancestorIdentifier=mgv1a001184m.g.IM62.v1.1; ...
24,ID=Migut.A00002.1.v2.0;Name,=Migut.A00002.1,;pacid=28938810;longest=1;Parent=Migut.A00002....
47,ID=Migut.A00002.2.v2.0;Name,=Migut.A00002.2,;pacid=28938811;longest=0;Parent=Migut.A00002....
...,...,...,...
489177,ID=Migut.O00655.1.v2.0;Name,=Migut.O00655.1,;pacid=28950645;longest=1;Parent=Migut.O00655....
489184,ID=Migut.O00656.v2.0;Name,=Migut.O00656,; Name=AHQT
489185,ID=Migut.O00656.1.v2.0;Name,=Migut.O00656.1,;pacid=28948024;longest=1;Parent=Migut.O00656....
489190,ID=Migut.O00657.v2.0;Name,=Migut.O00657,; Name=AHQT


0         ID=Migut.A00001.v2.0;Name=Migut.A00001_AHQT;an...
1         ID=Migut.A00001.1.v2.0;Name=Migut.A00001.1_AHQ...
23        ID=Migut.A00002.v2.0;Name=Migut.A00002_AHQT;an...
24        ID=Migut.A00002.1.v2.0;Name=Migut.A00002.1_AHQ...
47        ID=Migut.A00002.2.v2.0;Name=Migut.A00002.2_AHQ...
                                ...                        
489177    ID=Migut.O00655.1.v2.0;Name=Migut.O00655.1_AHQ...
489184    ID=Migut.O00656.v2.0;Name=Migut.O00656_AHQT; N...
489185    ID=Migut.O00656.1.v2.0;Name=Migut.O00656.1_AHQ...
489190    ID=Migut.O00657.v2.0;Name=Migut.O00657_AHQT; N...
489191    ID=Migut.O00657.1.v2.0;Name=Migut.O00657.1_AHQ...
Length: 61713, dtype: object

- in attribute column
- append ahqt or ahqn
- after Name=<entry>
- some rows do not have name
- if row is missing Name:
  - pass

# only mRNA and genes have names 

In [22]:
df_merge.feature_type.unique()

array(['gene', 'mRNA', 'exon', 'CDS', 'five_prime_UTR', 'three_prime_UTR'],
      dtype=object)

In [18]:
with fs.open('./YNP_RNA/pseudo.gff3', 'w') as fp: 
   fp.write('##gff-version 3\n##annot-version v2.0\n')
   df_merge.to_csv(fp, sep='\t', index=False, header=False)



